<a href="https://colab.research.google.com/github/noxouille/stable-diffusion-workflows/blob/main/custom_comfyui_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Git clone the repo and install the requirements. (ignore the pip errors about protobuf)

In [ ]:
#@title Environment Setup

from pathlib import Path

OPTIONS = {}

USE_GOOGLE_DRIVE = False  #@param {type:"boolean"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}
WORKSPACE = 'ComfyUI'
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI

if OPTIONS['USE_GOOGLE_DRIVE']:
    !echo "Mounting Google Drive..."
    %cd /

    from google.colab import drive
    drive.mount('/content/drive')

    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
    %cd /content/drive/MyDrive

![ ! -d $WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
  !echo -= Updating ComfyUI =-
  !git pull

!echo -= Install dependencies =-
!pip install xformers!=0.0.18 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu118 --extra-index-url https://download.pytorch.org/whl/cu117

# CONFIG DIR (not recommended to change unless you know what you're doing) modules/sd_models.py
models_dir = "./models/checkpoints/"
clips_dir = "./models/clip/"
clip_vision_dir = "./models/clip_vision/"
configs_dir = "./models/configs/"
controlnet_dir = "./models/controlnet/"
diffusers_dir = "./models/diffusers/"
embeddings_dir = "./models/embeddings/"
gligen_dir = "./models/gligen/"
hypernetworks_dir = "./models/hypernetworks/"
loras_dir = "./models/loras/"
style_models_dir = "./models/style_models/"
unet_dir = "./models/unet/"
upscale_models_dir = "./models/upscale_models/"
vae_dir = "./models/vae/"
vae_approx_dir = "./models/vae_approx/"
custom_nodes_dir = "./custom_nodes/"

# SDXL

In [ ]:
#@markdown ### **Checkpoints**
### I recommend these workflow examples: https://comfyanonymous.github.io/ComfyUI_examples/sdxl/
sd_xl_base_1_0 = True # @param {type:"boolean"}
sd_xl_refiner_1_0 = True # @param {type:"boolean"}
sd_xl_ReVision = True # @param {type:"boolean"}
DreamShaper_xl_1_0_alpha2 = True # @param {type:"boolean"}
ProtoVisionXL_beta_0_5_2_0_bakedVAE = False # @param {type:"boolean"}
unCLIP = False # @param {type:"boolean"}

if sd_xl_base_1_0:
  !wget -c https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors -P {models_dir}
if sd_xl_refiner_1_0:
  !wget -c https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0.safetensors -P {models_dir}
if DreamShaper_xl_1_0_alpha2:
  !wget -c https://civitai.com/api/download/models/126688 -O {models_dir}/DreamShaperXL_v1.0_alpha.safetensors
if ProtoVisionXL_beta_0_5_2_0_bakedVAE:
  !wget -c https://civitai.com/api/download/models/144229 -O {models_dir}/ProtoVisionXL_beta_v0.5.2.0_bakedVAE.safetensors
if sd_xl_ReVision:
  !wget -c https://huggingface.co/comfyanonymous/clip_vision_g/resolve/main/clip_vision_g.safetensors -P {clip_vision_dir}
if unCLIP:
  !wget -c https://huggingface.co/comfyanonymous/illuminatiDiffusionV1_v11_unCLIP/resolve/main/illuminatiDiffusionV1_v11-unclip-h-fp16.safetensors -P {models_dir}
  !wget -c https://huggingface.co/comfyanonymous/wd-1.5-beta2_unCLIP/resolve/main/wd-1-5-beta2-aesthetic-unclip-h-fp16.safetensors -P {models_dir}

#@markdown ### **VAE**
sdxl_vae = True # @param {type:"boolean"}
if sdxl_vae:
  !wget -c https://huggingface.co/stabilityai/sdxl-vae/resolve/main/sdxl_vae.safetensors -P {vae_dir}

#@markdown ### **LoRAs**
sd_xl_offset_example_lora_1_0 = True # @param {type:"boolean"}
if sd_xl_offset_example_lora_1_0:
  !wget -c https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_offset_example-lora_1.0.safetensors -P {loras_dir} #SDXL offset noise lora

#@markdown ### **ControlNet**
controlnet_sdxl = True # @param {type:"boolean"}
if controlnet_sdxl:
  !wget -c https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-canny-rank256.safetensors -P {controlnet_dir}
  !wget -c https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-depth-rank256.safetensors -P {controlnet_dir}
  !wget -c https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-recolor-rank256.safetensors -P {controlnet_dir}
  !wget -c https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-sketch-rank256.safetensors -P {controlnet_dir}

# Controlnet Preprocessor nodes by Fannovel16
controlnet_preproc_fannovel16 = True # @param {type:"boolean"}
if controlnet_preproc_fannovel16:
  !cd custom_nodes && git clone https://github.com/Fannovel16/comfyui_controlnet_aux.git; cd comfyui_controlnet_aux && python install.py;pip install -r requirements.txt

#@markdown ### **Upscaler**
NMKD_Siax_200k4x = True # @param {type:"boolean"}
UltraSharp4x = True # @param {type:"boolean"}
if NMKD_Siax_200k4x:
  !wget -c https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/4x_NMKD-Siax_200k.pth -P {upscale_models_dir}
if UltraSharp4x:
  !wget -c https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/4x-UltraSharp.pth -P {upscale_models_dir}


#@markdown ### **Custom Nodes**
custom_nodes = True # @param {type:"boolean"}
if custom_nodes:
  !git clone https://github.com/ltdrdata/ComfyUI-Manager.git {custom_nodes_dir}ComfyUI-Manager
  !git clone https://github.com/LucianoCirino/efficiency-nodes-comfyui.git {custom_nodes_dir}efficiency-nodes-comfyui
  !git clone https://github.com/ssitu/ComfyUI_NestedNodeBuilder.git {custom_nodes_dir}ComfyUI_NestedNodeBuilder
  !git clone https://github.com/bmad4ever/ComfyUI-Bmad-DirtyUndoRedo.git {custom_nodes_dir}ComfyUI-Bmad-DirtyUndoRedo
  !git clone https://github.com/Derfuu/Derfuu_ComfyUI_ModdedNodes {custom_nodes_dir}Derfuu_ComfyUI_ModdedNodes
  !git clone https://github.com/ltdrdata/ComfyUI-Impact-Pack.git {custom_nodes_dir}ComfyUI-Impact-Pack
  !git clone https://github.com/evanspearman/ComfyMath.git {custom_nodes_dir}ComfyMath
  !git clone https://github.com/marhensa/sdxl-recommended-res-calc.git {custom_nodes_dir}sdxl-recommended-res-calc
  !git clone https://github.com/ssitu/ComfyUI_UltimateSDUpscale {custom_nodes_dir}ComfyUI_UltimateSDUpscale --recursive

#@markdown ### **Workflows**
SeargeSDXL = True # @param {type:"boolean"}
if SeargeSDXL:
  !git clone https://github.com/SeargeDP/SeargeSDXL.git {custom_nodes_dir}SeargeSDXL

# @markdown ---

# **SD 1.5**

In [ ]:
#@markdown ### **Checkpoints**
SD1_5 = False # @param {type:"boolean"}
dreamshaper_v8 = False # @param {type:"boolean"}
NeverEndingDream_v1_22_baked_vae = False # @param {type:"boolean"}
Chilloutmix_Ni_pruned_fp32_fix = False # @param {type:"boolean"}
PerfectDeliberate_v4_0 = False # @param {type:"boolean"}
PerfectDeliberate_anime_v1_0 = False # @param {type:"boolean"}
fCAnimeMix_v3_0 = False # @param {type:"boolean"}
fCRadianceApex_v2_0 = False # @param {type:"boolean"}
fCBlendMix_v2_1 = False # @param {type:"boolean"}
RealisticVision_v5_1_VAE = False # @param {type:"boolean"}
MeichiDark_v4_5 = False # @param {type:"boolean"}
Fantexi_v0_9_beta = False # @param {type:"boolean"}
SakushiMix_finalVer = False # @param {type:"boolean"}
soapmix28D_v10 = False # @param {type:"boolean"}
AbyssOrangeMix2_hard = False # @param {type:"boolean"}
AOM3A1_orangemixs = False # @param {type:"boolean"}
AOM3A3_orangemixs = False # @param {type:"boolean"}
anything_v3_fp16_pruned = False # @param {type:"boolean"}
waifu_diffusion_illusion_fp16 = False # @param {type:"boolean"}

if SD1_5:
  !wget -c https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt -P {models_dir}
if dreamshaper_v8:
  !wget -c https://civitai.com/api/download/models/128713 -P {models_dir}
if NeverEndingDream_v1_22_baked_vae:
  !wget -c https://civitai.com/api/download/models/64094 -P {models_dir}
if Chilloutmix_Ni_pruned_fp32_fix:
  !wget -c https://civitai.com/api/download/models/11745 -P {models_dir}
if PerfectDeliberate_v4_0:
  !wget -c https://civitai.com/api/download/models/86698 -P {models_dir}
if PerfectDeliberate_anime_v1_0:
  !wget -c https://civitai.com/api/download/models/120036 -P {models_dir}
if fCAnimeMix_v3_0:
  !wget -c https://civitai.com/api/download/models/103277 -P {models_dir}
if fCRadianceApex_v2_0:
  !wget -c https://civitai.com/api/download/models/91270 -P {models_dir}
if fCBlendMix_v2_1:
  !wget -c https://civitai.com/api/download/models/90444 -P {models_dir}
if RealisticVision_v5_1_VAE:
  !wget -c https://civitai.com/api/download/models/130072 -P {models_dir}
if MeichiDark_v4_5:
  !wget -c https://civitai.com/api/download/models/147184 -P {models_dir}
if Fantexi_v0_9_beta:
  !wget -c https://civitai.com/api/download/models/95199 -P {models_dir}
if SakushiMix_finalVer:
  !wget -c https://civitai.com/api/download/models/133274 -P {models_dir}
if soapmix28D_v10:
  !wget -c https://huggingface.co/jtamph/soapmix/resolve/main/soapmix28D_v10.safetensors -P {models_dir}
if AbyssOrangeMix2_hard:
  !wget -c https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix2/AbyssOrangeMix2_hard.safetensors -P {models_dir}
if AOM3A1_orangemixs:
  !wget -c https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A1_orangemixs.safetensors -P {models_dir}
if AOM3A3_orangemixs:
  !wget -c https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A3_orangemixs.safetensors -P {models_dir}
if anything_v3_fp16_pruned:
  !wget -c https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/anything-v3-fp16-pruned.safetensors -P {models_dir}
if waifu_diffusion_illusion_fp16:  # Waifu Diffusion 1.5 (anime style SD2.x 768-v)
  !wget -c https://huggingface.co/waifu-diffusion/wd-1-5-beta3/resolve/main/wd-illusion-fp16.safetensors -P {models_dir}

#@markdown ### **VAE**
vae_ft_mse_840000_ema_pruned = True # @param {type:"boolean"}
orangemix_vae = False # @param {type:"boolean"}
kl_f8_anime2 = False # @param {type:"boolean"}
if vae_ft_mse_840000_ema_pruned:
  !wget -c https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors -P {vae_dir}
if orangemix_vae:
  !wget -c https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/VAEs/orangemix.vae.pt -P {vae_dir}
if kl_f8_anime2:
  !wget -c https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt -P {vae_dir}

#@markdown ### **LoRAs**
theovercomer8sContrastFix_sd21768 = False # @param {type:"boolean"}
theovercomer8sContrastFix_sd15 = False # @param {type:"boolean"}
LowRA = False # @param {type:"boolean"}
if theovercomer8sContrastFix_sd21768:
  !wget -c https://civitai.com/api/download/models/10350 -O {loras_dir}theovercomer8sContrastFix_sd21768.safetensors #theovercomer8sContrastFix SD2.x 768-v
if theovercomer8sContrastFix_sd15:
  !wget -c https://civitai.com/api/download/models/10638 -O {loras_dir}theovercomer8sContrastFix_sd15.safetensors #theovercomer8sContrastFix SD1.x
if LowRA:
  !wget -c https://civitai.com/api/download/models/63006 -O {loras_dir}LowRA.safetensors

#@markdown ### **ControlNet**
controlnet_sd_1_5 = False # @param {type:"boolean"}
if controlnet_sd_1_5:
  !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_ip2p_fp16.safetensors -P {controlnet_dir}
  !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_shuffle_fp16.safetensors -P {controlnet_dir}
  !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_canny_fp16.safetensors -P {controlnet_dir}
  !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors -P {controlnet_dir}
  !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors -P {controlnet_dir}
  !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_lineart_fp16.safetensors -P {controlnet_dir}
  !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_mlsd_fp16.safetensors -P {controlnet_dir}
  !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_normalbae_fp16.safetensors -P {controlnet_dir}
  !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_openpose_fp16.safetensors -P {controlnet_dir}
  !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_scribble_fp16.safetensors -P {controlnet_dir}
  !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_seg_fp16.safetensors -P {controlnet_dir}
  !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_softedge_fp16.safetensors -P {controlnet_dir}
  !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors -P {controlnet_dir}
  !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11u_sd15_tile_fp16.safetensors -P {controlnet_dir}

#@markdown ### **Upscaler**
esrgan = False # @param {type:"boolean"}
if esrgan:
  !wget -c https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P {upscale_models_dir}
  !wget -c https://huggingface.co/sberbank-ai/Real-ESRGAN/resolve/main/RealESRGAN_x2.pth -P {upscale_models_dir}
  !wget -c https://huggingface.co/sberbank-ai/Real-ESRGAN/resolve/main/RealESRGAN_x4.pth -P {upscale_models_dir}


#@markdown ### **T2I-Adapter**
t2i_adapter = False # @param {type:"boolean"}
if t2i_adapter:
  !wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_depth_sd14v1.pth -P {controlnet_dir}
  !wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_seg_sd14v1.pth -P {controlnet_dir}
  !wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_sketch_sd14v1.pth -P {controlnet_dir}
  !wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_keypose_sd14v1.pth -P {controlnet_dir}
  !wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_openpose_sd14v1.pth -P {controlnet_dir}
  !wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_color_sd14v1.pth -P {controlnet_dir}
  !wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_canny_sd14v1.pth -P {controlnet_dir}

t2i_styles_model = False # @param {type:"boolean"}
if t2i_styles_model:
  !wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_style_sd14v1.pth -P {style_models_dir}

#@markdown ### **CLIPVision model (needed for styles model)**
CLIPVision_model = False # @param {type:"boolean"}
if CLIPVision_model:
  !wget -c https://huggingface.co/openai/clip-vit-large-patch14/resolve/main/pytorch_model.bin -O {clip_vision_dir}clip_vit14.bin





# GLIGEN
#!wget -c https://huggingface.co/comfyanonymous/GLIGEN_pruned_safetensors/resolve/main/gligen_sd14_textbox_pruned_fp16.safetensors -P ./models/gligen/


# @markdown ---

# **SD 2.1**

In [ ]:
#@markdown # **SD 2.1**
#@markdown ### **Checkpoints**
v2_1_512_ema_pruned = False # @param {type:"boolean"}
v2_1_768_ema_pruned = False # @param {type:"boolean"}
if v2_1_512_ema_pruned:
  !wget -c https://huggingface.co/stabilityai/stable-diffusion-2-1-base/resolve/main/v2-1_512-ema-pruned.safetensors -P {models_dir}
if v2_1_768_ema_pruned:
  !wget -c https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.safetensors -P {models_dir}


# Run ComfyUI

In [ ]:
run_option = 'cloudflared (Recommended Way)' # @param ["cloudflared (Recommended Way)", "localtunnel", "colab iframe"]
print('Running ComfyUI with ', run_option)
if run_option=="cloudflared (Recommended Way)":
  !wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
  !dpkg -i cloudflared-linux-amd64.deb

  import subprocess
  import threading
  import time
  import socket
  import urllib.request

  def iframe_thread(port):
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
          break
        sock.close()
    print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

    p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    for line in p.stderr:
      l = line.decode()
      if "trycloudflare.com " in l:
        print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
      #print(l, end='')


  threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

  !python main.py --dont-print-server


if run_option=="localtunnel":
  !npm install -g localtunnel

  import subprocess
  import threading
  import time
  import socket
  import urllib.request

  def iframe_thread(port):
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
          break
        sock.close()
    print("\nComfyUI finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)\n")

    print("The password/enpoint ip for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
    p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
    for line in p.stdout:
      print(line.decode(), end='')


  threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

  !python main.py --dont-print-server

if run_option == "colab iframe":
  import threading
  import time
  import socket
  def iframe_thread(port):
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
          break
        sock.close()
    from google.colab import output
    output.serve_kernel_port_as_iframe(port, height=1024)
    print("to open it in a window you can open this link here:")
    output.serve_kernel_port_as_window(port)

  threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

  !python main.py --dont-print-server

#@markdown (use colab iframe only in case the previous way with localtunnel doesn't work)<br>
#@markdown You should see the ui appear in an iframe. If you get a 403 error, it's your firefox settings or an extension that's messing things up.<br>
#@markdown If you want to open it in another window use the link.<br><br>
#@markdown Note that some UI features like live image previews won't work because the colab iframe blocks websockets.
